In [1]:
import pandas as pd
import numpy as np
import pydata_google_auth
import plotly.express as px

SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform',
    'https://www.googleapis.com/auth/drive',
]

credentials = pydata_google_auth.get_user_credentials(
    SCOPES,
    auth_local_webserver=True,
)

In [2]:
query = '''
WITH finalized_dataset AS
(
  WITH unfiltered_dataset AS
  (
    SELECT *
    FROM bigquery-public-data.chicago_taxi_trips.taxi_trips 
    WHERE 
      unique_key IS NOT NULL
      AND taxi_id IS NOT NULL
      AND trip_start_timestamp IS NOT NULL
      AND trip_start_timestamp IS NOT NULL
      AND trip_end_timestamp IS NOT NULL
      AND trip_seconds IS NOT NULL
      AND trip_seconds <> 0 -- Outlier
      AND trip_miles IS NOT NULL
      AND trip_miles <> 0 -- Outlier
      AND pickup_census_tract IS NOT NULL
      AND dropoff_census_tract IS NOT NULL
      AND pickup_community_area IS NOT NULL
      AND dropoff_community_area IS NOT NULL
      AND fare IS NOT NULL
      AND fare <> 0 -- Outlier
      AND tips IS NOT NULL
      AND tolls IS NOT NULL
      AND extras IS NOT NULL
      AND trip_total IS NOT NULL
      AND trip_total <> 0 -- Outlier
      AND payment_type IS NOT NULL
      AND company IS NOT NULL
      AND pickup_latitude IS NOT NULL
      AND pickup_longitude IS NOT NULL
      AND pickup_location IS NOT NULL
      AND dropoff_latitude IS NOT NULL
      AND dropoff_longitude IS NOT NULL
      AND dropoff_location IS NOT NULL
      AND TIMESTAMP_DIFF(trip_end_timestamp, trip_start_timestamp, SECOND) = trip_seconds
  )

  , unfiltered_dataset_2 AS
  (
    SELECT
    company,
    COUNT(unique_key) cnt_trip
    FROM unfiltered_dataset
    GROUP BY 1
    order by 2 desc
    limit 10
  )

  SELECT * FROM unfiltered_dataset
  WHERE company IN
  (
    SELECT company FROM unfiltered_dataset_2
  )
)

, transform AS
(
  SELECT
  pickup_longitude long,
  pickup_latitude lat,
  COUNT(*) AS count,
  FROM finalized_dataset
  GROUP BY 1,2
  -- HAVING COUNT(*) > 1
)

SELECT * FROM transform
'''

df = pd.read_gbq(query,
                    project_id='data-gojek-access',
                    dialect="standard",
                    credentials=credentials)
df

In [ ]:
df.head()

In [ ]:
fig = px.density_mapbox(df, lat='lat', lon='long', z='count',
                        mapbox_style="stamen-terrain")
 
fig